# LangChain Examples
LangChain is a framework for developing applications powered by Large Language Models (LLMs). It enables applications that:
- Are context-aware: connect a language model to sources of context (prompt instructions, few shot examples, content to ground its response in, etc.)
- Reason: rely on a language model to reason (about how to answer based on provided context, what actions to take, etc.)

## Lesson Examples
- Prompt Templates
- Prompt Chains
- Memory
- Tools

### Setup
ChatGPT is used as the LLM for these examples, but supports a ton of LLMs.

In [144]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

### Prompt Templates
With LangChain you can easily create templates for your prompts.

In [145]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0.0, model="gpt-3.5-turbo")

template = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

# create prompt template
prompt_template = ChatPromptTemplate.from_template(template)

##### Create Template Variables

In [146]:
style = """American English \
in a calm and respectful tone
"""

customer_email = """
抱歉，這次購買電視的體驗讓我非常失望。\
首先，產品的品質並不符合我的期望，\
屏幕有明顯的問題，影響觀看效果。其次，\
客戶服務也十分不滿意，解決問題的速度很慢，\
而且態度不夠友好。總的來說，這次購買讓我感到非常不滿，\
我不會推薦這個品牌給任何人。
"""

##### Initialize template varibales

In [147]:
customer_messages = prompt_template.format_messages(
                    style=style,
                    text=customer_email)

##### Send Request & Print Response

In [148]:
customer_response = chat.invoke(customer_messages)
print(customer_response.content)

I'm sorry, but my experience with purchasing this TV has been very disappointing. First of all, the quality of the product did not meet my expectations, as the screen has obvious issues that affect the viewing experience. Additionally, I was very dissatisfied with the customer service - the problem-solving process was slow and the attitude was not friendly enough. Overall, this purchase has left me feeling very unsatisfied, and I would not recommend this brand to anyone.


### Prompt Chains
At times we want to take the response from one prompt and use it in another. We can acomplish this with **SequentialChain and LLMChain**.

In [149]:
from langchain.chains import SimpleSequentialChain
from langchain.chains import LLMChain

llm = ChatOpenAI(temperature=0.9, model="gpt-3.5-turbo")

product = "Queen Size Sheet Set"

##### Create first prompt in the chain

In [150]:
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

##### Create second prompt in the chain

In [151]:
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 word description for the following \
    company:{company_name}"
)

# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

##### Link the chains together with a SimpleSequentialChain

In [152]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

##### Send the request & print response

In [153]:
overall_simple_chain.invoke(product)



> Entering new SimpleSequentialChain chain...
Regal Linens
Regal Linens provides luxurious and high-quality bedding and linens to elevate your home decor and create a stylish ambiance.

> Finished chain.


{'input': 'Queen Size Sheet Set',
 'output': 'Regal Linens provides luxurious and high-quality bedding and linens to elevate your home decor and create a stylish ambiance.'}

### Memory
LLMs are stateless. This means they do not store the conversation history. LangChain provides an API that you can use to store conversation history.

In [154]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(temperature=0.0, model="gpt-3.5-turbo")
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [155]:
conversation.predict(input="Hi, my name is Andrew")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.


"Hello Andrew! It's nice to meet you. How can I assist you today?"

In [156]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'1+1 equals 2. Is there anything else you would like to know?'

In [157]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 equals 2. Is there anything else you would like to know?
Human: What is my name?
AI:

> Finished chain.


'Your name is Andrew. Is there anything else you would like to know, Andrew?'

### Tools
LangChain is equipped with tons of tools that you can use to integrate within your LLM. [See More](https://python.langchain.com/docs/integrations/tools)

##### Example Wikidata

In [158]:
from langchain_community.tools.wikidata.tool import WikidataAPIWrapper, WikidataQueryRun

wikidata = WikidataQueryRun(api_wrapper=WikidataAPIWrapper())

print(wikidata.run("Alan Turing"))

Result Q7251:
Label: Alan Turing
Description: English computer scientist (1912–1954)
Aliases: Alan M. Turing, Alan Mathieson Turing, Turing, Alan Mathison Turing
instance of: human
country of citizenship: United Kingdom
occupation: computer scientist, mathematician, university teacher, cryptographer, logician, statistician, marathon runner, artificial intelligence researcher
sex or gender: male
date of birth: 1912-06-23
date of death: 1954-06-07
sport: athletics
place of birth: Maida Vale, Warrington Lodge
educated at: King's College, Princeton University, Sherborne School, Hazlehurst Community Primary School
employer: Victoria University of Manchester, Government Communications Headquarters, University of Cambridge, National Physical Laboratory
place of death: Wilmslow
field of work: cryptanalysis, computer science, mathematics, logic, cryptography
cause of death: cyanide poisoning
notable work: On Computable Numbers, with an Application to the Entscheidungsproblem, Computing Machiner

##### Example Youtube Search

In [159]:
from langchain.tools import YouTubeSearchTool

tool = YouTubeSearchTool()

tool.run("lex friedman")

"['https://www.youtube.com/watch?v=PgGKhsWhUu8&pp=ygUMbGV4IGZyaWVkbWFu', 'https://www.youtube.com/watch?v=zgbFJiGHVAc&pp=ygUMbGV4IGZyaWVkbWFu']"

##### Example PubMed

In [160]:
from langchain_community.tools.pubmed.tool import PubmedQueryRun

tool = PubmedQueryRun()

tool.invoke("What causes lung cancer?")

"Published: 2024-02-19\nTitle: Investigating the potential causal association between consumption of green tea and risk of lung cancer: a study utilizing Mendelian randomization.\nCopyright Information: Copyright © 2024 Lu, Lin, Jiang, Gao, Shen, Yang, Lin and Kang.\nSummary::\nBACKGROUND: Lung cancer is the most common global cancer in terms of incidence and mortality. Its main driver is tobacco smoking. The identification of modifiable risk factors isa public health priority. Green tea consumption has been examined in epidemiological studies, with inconsistent findings. Thus, we aimed to apply Mendelian randomization to clarify any causal link between green tea consumption and the risk of lung cancer.\nMETHODS: We utilized a two-sample Mendelian randomization (MR) approach. Genetic variants served as instrumental variables. The goal was to explore a causal link between green tea consumption and different lung cancer types. Green tea consumption data was sourced from the UK Biobank da